In [1]:
import glob
import csv
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
#import funcy
#from funcy import print_durations
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
returns_df = pd.read_csv('/Users/sakshigupta/Downloads/2010-22/return_all.csv',index_col=0)
returns_df = returns_df.dropna(axis=1,how='all')
equity_idx = returns_df.columns

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [3]:
files = glob.glob('/Users/sakshigupta/Downloads/2010-22/alpha*')
names = list(map(lambda nm: nm[8:16], glob.glob('2010-22/alpha*')))
alpha_dict = {}
for name, file in zip(names,files):
    try:
        df = pd.read_csv(file,index_col=0).loc[:,equity_idx]
        df = df.transpose().fillna(df.mean(axis=1),axis=0)\
            .transpose().dropna(axis=0,how='all').shift(1).iloc[1:]
        df['Alpha'] = name
        df = df.set_index([df.index, 'Alpha'])
        alpha_dict[name] = df
    except Exception as e:
        print(name)
        print(e)

In [4]:
full_df = pd.concat(list(alpha_dict.values()),axis=0)
full_df = full_df.sort_index(axis=0,level=1,kind='mergesort').sort_index(axis=0,level=0,kind='mergesort')
full_df1 = full_df.stack().unstack(level=1)
full_df1.index.set_names(['Dates', 'Equity'], inplace=True)

In [5]:
full_df1 = full_df1.fillna(0)
returns_df = returns_df.fillna(0)

In [6]:
full_df1=full_df1.drop(columns=['alpha040'])
full_df1=full_df1.drop(columns=['alpha027'])

In [7]:
alphas_df_train, returns_df_train = full_df1.loc['2015-01-01':'2021-07-01'].copy(), returns_df.loc['2015-01-01':'2021-07-01'].copy()
alphas_df_test, returns_df_test = full_df1.loc['2021-07-01':], returns_df.loc['2021-07-01':]

In [8]:
alphas_df_train = alphas_df_train.reorder_levels(['Equity','Dates']).sort_index()
alphas_df_test = alphas_df_test.reorder_levels(['Equity','Dates']).sort_index()

In [18]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
regr = linear_model.LinearRegression()

c = alphas_df_train.index.get_level_values(0)
d = alphas_df_train.index.get_level_values(1)

model_fit={}
for i in c:
    x = alphas_df_train.loc[i,:]
    y = returns_df_train.loc[:,i]
    model = regr.fit(x, y)
    model_fit[i] = model

KeyboardInterrupt: 

In [ ]:
pred_y = np.zeros((returns_df.shape[0]-1,returns_df.shape[1]-1))
for i in c:
    for j in d:
        try:
            x = np.array(full_df1 .loc[i,j]).reshape(1,-1)
            y = returns_df.loc[j,i]
            pred_y[j,i] = model_fit[i].predict(full_df1.loc[i+1,j+1]) 
        except KeyError:
            break 

In [75]:
alphas_df_train.index.get_level_values(1)

Index(['2015-01-02', '2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08',
       '2015-01-09', '2015-01-12', '2015-01-13', '2015-01-14', '2015-01-15',
       ...
       '2021-06-18', '2021-06-21', '2021-06-22', '2021-06-23', '2021-06-24',
       '2021-06-25', '2021-06-28', '2021-06-29', '2021-06-30', '2021-07-01'],
      dtype='object', name='Dates', length=4844884)

In [101]:
np.array(alphas_df_train.loc['0111145D UN Equity','2015-01-02']).reshape(1,-1)
# loc[:,'2015-01-02']

# .loc['0111145D UN Equity']

array([[ 4.13532653e-01,  0.00000000e+00,  0.00000000e+00,
        -5.28956916e+00, -1.81246882e-01,  0.00000000e+00,
         1.75617002e+00, -7.10970505e-01, -4.79795575e+03,
        -4.79786610e+03,  8.94079283e-01,  5.38192870e+00,
        -4.05716777e-01,  7.76568664e-02, -1.51080550e+00,
        -5.50781483e-01, -1.45083075e-01, -3.36792561e-01,
         5.15896015e-01, -1.58700322e-01, -1.30913688e-01,
         0.00000000e+00, -1.70932408e+05,  6.61167858e-01,
        -0.00000000e+00,  1.00000000e+00,  1.45789719e-04,
         3.41965642e+00,  9.75728850e-02,  1.39588773e+00,
         9.22905193e-03,  6.85805444e-01,  6.29041435e-01,
         1.72111624e+03,  3.13705826e+00,  1.08779960e+00,
        -1.81374230e-01, -7.92280374e-01,            -inf,
        -1.93853566e+03,  2.10866189e+00,  5.78335632e+01,
        -0.00000000e+00,  9.18804467e-02, -2.36427832e-01,
         8.75910737e-01, -4.77573705e+03, -7.03694029e-01]])

In [ ]:
alphas_df_train.loc[i,j] 

In [40]:
returns_df_train.head(10)

,EIX UN Equity,MSFG UW Equity,OHAI UW Equity,OFLX UQ Equity,GAIN UW Equity,APAGF UR Equity,HTLD UW Equity,FRO UN Equity,AA UN Equity,AXP UN Equity,...,LXRX UQ Equity,TSYS UQ Equity,ARNA UQ Equity,1797629D UQ Equity,ACTG UQ Equity,HTWR UQ Equity,BCPC UW Equity,GPRE UQ Equity,COWN UQ Equity,1664784D UQ Equity
Dates,,,,,,,,,,,,,,,,,,,,,
2018-10-01,-0.1921,0.0,0.3300,-1.0399,-0.2625,0.0,-1.6726,4.9914,1.1386,1.3241,...,-1.4995,0.0,0.2608,0.0,0.3125,0.0,-1.1419,0.2907,1.4724,0.0
2018-10-02,1.7321,0.0,0.0000,0.0994,-0.5263,0.0,-2.3711,1.1475,1.7132,0.4541,...,-5.7088,0.0,-0.9753,0.0,0.0000,0.0,-1.8590,0.9855,3.4462,0.0
2018-10-03,-0.4075,0.0,3.2895,0.1277,-0.1764,0.0,1.0560,5.0243,3.2002,0.0554,...,0.7064,0.0,4.1585,0.0,-0.9346,0.0,-0.0368,3.8462,3.2145,0.0
2018-10-04,0.3653,0.0,-3.8217,0.0567,-2.4735,0.0,-2.5601,-1.0802,0.5129,-0.2961,...,-2.5050,0.0,-9.2877,0.0,-0.6289,0.0,-1.4626,-3.2062,-1.6421,0.0
2018-10-05,0.8880,0.0,-3.3113,-2.2373,-0.2717,0.0,-2.3592,1.2480,-2.4356,-0.4734,...,-2.5694,0.0,-7.0188,0.0,1.2658,0.0,-1.3630,0.5711,0.0576,0.0
2018-10-08,0.9091,0.0,6.1644,0.2607,-0.8174,0.0,0.7688,-2.1572,-7.8935,-0.2238,...,-1.5823,0.0,-1.1211,0.0,0.0000,0.0,-0.5963,4.8836,0.6329,0.0
2018-10-09,0.7436,0.0,-0.6452,0.4334,-1.4652,0.0,0.6540,4.7244,-2.8394,-0.3084,...,-1.8221,0.0,0.1764,0.0,-1.5625,0.0,-0.9045,-4.8728,-0.5718,0.0
2018-10-10,-0.5252,0.0,-2.5974,-1.7261,-0.0929,0.0,-2.3822,-4.5113,-4.1977,-2.8971,...,-4.4760,0.0,-1.9366,0.0,-1.5873,0.0,-3.4877,-8.3665,-5.7504,0.0
2018-10-11,-2.3830,0.0,-1.3333,-3.8056,-0.5581,0.0,-1.4975,-3.9370,-1.0815,-1.9214,...,-1.4857,0.0,1.5902,0.0,-1.6129,0.0,-1.9214,17.4534,-4.8810,0.0


In [98]:
returns_df_train.loc['2015-01-02','EIX UN Equity']

0.0458

In [16]:
returns_df_train.loc[:,'EIX UN Equity']

Dates
2015-01-02    0.0458
2015-01-05   -0.2442
2015-01-06    0.3826
2015-01-07    1.6768
2015-01-08    0.5547
               ...  
2021-06-25    3.1114
2021-06-28    1.1053
2021-06-29   -0.3644
2021-06-30    0.6966
2021-07-01    0.1557
Name: EIX UN Equity, Length: 1639, dtype: float64

-17.615436670385062